In [1]:
import gpxpy
import pandas as pd

import matplotlib.pyplot as plt

import gzip
from lxml import etree


import os

- Point at directory of new export
- Pull in 'activities'
- Build a top-level activity dataframe
- For run, walk and ride create 3 lists of files
- For each activity: 
    - create a dataframe of the time, heartrate, speed, lat/long and output


In [2]:
main_dir = '../strava_data_dumps/STRAVA+export_8029714/activities/'

## Generate a dataframe

In [10]:

# Load a GPX file
with open(f'{main_dir}/261971610.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

points = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation,
                'time': point.time,
                'speed': point.speed,
                'heart_rate': 'NaN'
            })

df = pd.DataFrame(points)
df

,latitude,longitude,elevation,time,speed,heart_rate
0,54.733356,-1.584758,69.5,2015-03-01 17:35:32+00:00,None,NaN
1,54.733352,-1.584782,69.5,2015-03-01 17:35:33+00:00,None,NaN
2,54.733340,-1.584827,69.6,2015-03-01 17:35:35+00:00,None,NaN
3,54.733333,-1.584880,69.7,2015-03-01 17:35:38+00:00,None,NaN
4,54.733329,-1.584934,69.7,2015-03-01 17:35:41+00:00,None,NaN
...,...,...,...,...,...,...
84,54.733093,-1.585461,71.2,2015-03-01 17:47:46+00:00,None,NaN
85,54.733070,-1.585437,71.3,2015-03-01 17:47:48+00:00,None,NaN
86,54.733049,-1.585394,71.3,2015-03-01 17:47:51+00:00,None,NaN
87,54.733056,-1.585450,71.3,2015-03-01 17:47:55+00:00,None,NaN


In [ ]:
## Generate elevation map

# Optional: calculate distance for x-axis (if time/distance based is preferred)
df['time'] = pd.to_datetime(df['time'])
df['elapsed_time_min'] = (df['time'] - df['time'].min()).dt.total_seconds() / 60

# Plot elevation over time
plt.figure(figsize=(12, 5))
plt.plot(df['elapsed_time_min'], df['elevation'], color='darkorange')
plt.xlabel('Time Elapsed (minutes)')
plt.ylabel('Elevation (m)')
plt.title('Elevation Profile')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

with gzip.open('activity.tcx.gz', 'rb') as f:
    tree = etree.parse(f)
    root = tree.getroot()

    for elem in root.iter():
        print(elem.tag, elem.text)


In [ ]:
import gzip
from fitparse import FitFile

all_records = []

with gzip.open(f'{main_dir}/13410295724.fit.gz', 'rb') as f:
    fitfile = FitFile(f)
    for record in fitfile.get_messages('record'):
        data = {d.name: d.value for d in record}
        #data['source_file'] = file  # Optional: track which file it came from
        all_records.append(data)


In [ ]:
df = pd.DataFrame(all_records)

In [ ]:
df

# Loop everything into a df

In [12]:
file_path = f'{main_dir}/13410295724.fit.gz'

with gzip.open(file_path, 'rb') as f:
    fitfile = FitFile(f)
    for record in fitfile.get_messages('record'):
        data = {d.name: d.value for d in record}
        data['source_file'] = file  # Optional: track which file it came from
        all_records.append(data)

# Convert to DataFrame
df = pd.DataFrame(all_records)

def semicircles_to_degrees(semicircles):
    
    """
    Converts lat/long from the fit format of semicircle --> normal lat/longs
    """

    return semicircles * (180 / 2**31)

df['LAT'] = df['position_lat'].apply(semicircles_to_degrees)
df['LONG'] = df['position_long'].apply(semicircles_to_degrees)
#df.to_csv('GPS_route.csv')

In [13]:
df

,distance,timestamp,source_file,enhanced_altitude,enhanced_speed,gps_accuracy,position_lat,position_long,speed,heart_rate,LAT,LONG
0,0.00,2023-10-18 11:21:02,10778596247.fit.gz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2023-10-18 11:21:02,10778596247.fit.gz,41.8,1.354,3.0,653310815.0,-18523787.0,1.354,NaN,54.759880,-1.552646
2,NaN,2023-10-18 11:21:03,10778596247.fit.gz,41.8,1.438,3.0,653310656.0,-18523762.0,1.438,NaN,54.759867,-1.552644
3,NaN,2023-10-18 11:21:04,10778596247.fit.gz,41.8,1.478,2.0,653310490.0,-18523729.0,1.478,105.0,54.759853,-1.552641
4,NaN,2023-10-18 11:21:05,10778596247.fit.gz,41.8,1.495,2.0,653310319.0,-18523706.0,1.495,NaN,54.759838,-1.552639
...,...,...,...,...,...,...,...,...,...,...,...,...
24544,NaN,2024-10-05 08:05:01,13359710315.fit.gz,47.8,0.061,1.0,653308171.0,-18503315.0,0.061,NaN,54.759658,-1.550930
24545,NaN,2024-10-05 08:05:02,13359710315.fit.gz,47.6,0.051,2.0,653308176.0,-18503308.0,0.051,NaN,54.759659,-1.550929
24546,20856.27,2024-10-05 08:05:03,13359710315.fit.gz,47.6,0.032,2.0,653308180.0,-18503304.0,0.032,NaN,54.759659,-1.550929
24547,20856.29,2024-10-05 08:05:04,13359710315.fit.gz,47.6,0.018,2.0,653308182.0,-18503301.0,0.018,NaN,54.759659,-1.550929


In [ ]:
import os
import gzip
import pandas as pd
from fitparse import FitFile

all_records = []

# Walk through directories and subdirectories
for root, dirs, files in os.walk(main_dir):
    for file in files:
        if file.endswith('.fit.gz'):
            file_path = os.path.join(root, file)
            print(file_path)
            with gzip.open(file_path, 'rb') as f:
                fitfile = FitFile(f)
                for record in fitfile.get_messages('record'):
                    data = {d.name: d.value for d in record}
                    data['source_file'] = file  # Optional: track which file it came from
                    all_records.append(data)

# Convert to DataFrame
df = pd.DataFrame(all_records)

def semicircles_to_degrees(semicircles):
    
    """
    Converts lat/long from the fit format of semicircle --> normal lat/longs
    """

    return semicircles * (180 / 2**31)

df['LAT'] = df['position_lat'].apply(semicircles_to_degrees)
df['LONG'] = df['position_long'].apply(semicircles_to_degrees)
df.to_csv('GPS_route.csv')

../strava_data_dumps/STRAVA+export_8029714/activities/10778596247.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/11061016795.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/12759412255.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/12852566380.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/13282848920.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/8920586120.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/9950417558.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/10751797773.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/11963115478.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/11423600841.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/11195088990.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/11114999386.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/9459247241.fit.gz
../strava_data_dumps/STRAVA+export_8029714/activities/

KeyboardInterrupt: 

In [ ]:
## Generate a Map

import folium
# Get the central point of the route to center the map
start_coords = (df['latitude'].mean(), df['longitude'].mean())

# Create the map
m = folium.Map(location=start_coords, zoom_start=13)

# Draw the route as a line
folium.PolyLine(
    list(zip(df['latitude'], df['longitude'])),
    color='blue',
    weight=4,
    opacity=0.8
).add_to(m)

# Optionally, mark start and end points
folium.Marker(
    location=[df.iloc[0]['latitude'], df.iloc[0]['longitude']],
    popup='Start',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[df.iloc[-1]['latitude'], df.iloc[-1]['longitude']],
    popup='End',
    icon=folium.Icon(color='red')
).add_to(m)

# Show the map in Jupyter or export to HTML
m.save("route_map.html")